In [117]:
import numpy as np
import pandas as pd
from mlx_lm import load, generate
import re

In [116]:
df = pd.read_csv('Test_dataset(Real).csv')
df.dropna(inplace=True)
df = df.drop(columns=['Sr No.'])
df['Label'] = 'True'
df.head()

,Headline,Source,Label
0,NASA’s Perseverance rover finds its first poss...,sciencenews.org,True
1,Sepsis tests take days putting patients at ri...,sciencenews.org,True
2,Nasa's DART asteroid unlocks complex history o...,https://timesofindia.indiatimes.com/,True
3,Say goodbye to back pain patients go for adva...,https://timesofindia.indiatimes.com/,True
4,Neurodivergent children more likely to develop...,https://timesofindia.indiatimes.com/,True


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 344 entries, 0 to 348
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  344 non-null    object
 1   Source    344 non-null    object
 2   Label     344 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


In [111]:
model, tokenizer = load("microsoft/Phi-3-mini-4k-instruct")

Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 111962.94it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [112]:
def generate_fake_headline(real_headline):
    query = f'''
    <|user|>
    Given the true headline below ,you have to make changes to it using a combination of
    Sentence negation, Number swaps, Replacing Named Entities and Noise Injection to generate a fake news headline. 
    The output should be only the generated fake news headline for evaluating my fake news model.

    True Headline : {real_headline} <|end|>
    <|assistant|>'''
    response = generate(model, tokenizer, prompt=query)
    fake_headline =response.replace('<|end|>','')
    fake_headline =fake_headline.replace('Fake News Headline:','')
    print('Real Headline : ',real_headline)
    print(f'Generated Headline : {fake_headline}\n')
    return fake_headline

new_rows = []
for headline in df['Headline']:
    fake_headline = generate_fake_headline(headline)
    new_rows.append({'Headline': fake_headline, 'Source': 'Synthetically Generated', 'Label': 'False'})

# Append the new rows to the original DataFrame
df_synthetic = pd.DataFrame(new_rows)
df = pd.concat([df, df_synthetic], ignore_index=True)

Real Headline :  NASA’s Perseverance rover finds its first possible hint of ancient life on Mars
Generated Headline : NASA's Perseverance rover finds no signs of ancient life on Mars.

Real Headline :  Sepsis tests take days  putting patients at risk. A new method may cut wait time
Generated Headline : "Sepsis tests now take days, putting patients at risk. A new method may increase wait time."

Real Headline :  Nasa's DART asteroid unlocks complex history of twin space rock targets; see pics
Generated Headline : NASA's DART asteroid reveals simple past of lone space rock targets; check out images

Real Headline :  Say goodbye to back pain  patients go for advanced endoscopy spine surgery for sciatica
Generated Headline : "Say hello to back pain, patients avoid advanced endoscopy spine surgery for sciatica"

Real Headline :  Neurodivergent children more likely to develop chronic fatigue as teen  study finds
Generated Headline : "Study finds neurotypical teens less prone to chronic fatig

In [113]:
df

,Headline,Source,Label
0,NASA’s Perseverance rover finds its first poss...,sciencenews.org,True
1,Sepsis tests take days putting patients at ri...,sciencenews.org,True
2,Nasa's DART asteroid unlocks complex history o...,https://timesofindia.indiatimes.com/,True
3,Say goodbye to back pain patients go for adva...,https://timesofindia.indiatimes.com/,True
4,Neurodivergent children more likely to develop...,https://timesofindia.indiatimes.com/,True
...,...,...,...
683,"""Google's parent company faces UK investigatio...",Synthetically Generated,False
684,"""Intel announces hiring spree to fund recovery...",Synthetically Generated,False
685,"""Ola Electric IPO grey market premium skyrocke...",Synthetically Generated,False
686,"""Japan's Nikkei 225 Index Soars 12.4%, Its Big...",Synthetically Generated,False


In [140]:
df

,Headline,Source,Label
0,NASA’s Perseverance rover finds its first poss...,sciencenews.org,True
1,Sepsis tests take days putting patients at ri...,sciencenews.org,True
2,Nasa's DART asteroid unlocks complex history o...,https://timesofindia.indiatimes.com/,True
3,Say goodbye to back pain patients go for adva...,https://timesofindia.indiatimes.com/,True
4,Neurodivergent children more likely to develop...,https://timesofindia.indiatimes.com/,True
...,...,...,...
683,"""Google's parent company faces UK investigatio...",Synthetically Generated,False
684,"""Intel announces hiring spree to fund recovery...",Synthetically Generated,False
685,"""Ola Electric IPO grey market premium skyrocke...",Synthetically Generated,False
686,"""Japan's Nikkei 225 Index Soars 12.4%, Its Big...",Synthetically Generated,False


In [151]:
df.to_csv('Test_dataset.csv', index=False)